In [ ]:
# Basic Libraries
import numpy as np
import pandas as pd

# Visualization libraries
import matplotlib.pyplot as plt

#Evaluation library
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# Deep Learning libraries
import tensorflow as tf
from tensorflow.keras import layers
import keras
from keras.models import Sequential
from keras.layers.core import Dense,Activation,Dropout
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
#Digit MNIST dataset
(X_train_digit, y_train_digit), (X_test_digit, y_test_digit) = mnist.load_data()

#Fashion MNIST dataset
fashion_train=pd.read_csv("./input_data/train.csv")
fashion_test=pd.read_csv("./input_data/test.csv")

In [ ]:
fashion_train.head()

In [ ]:
X_train_fashion = fashion_train.drop('label', axis=1)
y_train_fashion = fashion_train['label']
X_test_fashion = fashion_test.drop('label', axis=1)
y_test_fashion = fashion_test['label']

In [ ]:
#Reshaping the dataset
x_train_reshape = X_train_fashion.values.reshape(-1,28,28)
x_test_reshape = X_test_fashion.values.reshape(-1,28,28)

#Names of clothing accessories in order 
col_names = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']

#Visualizing the images
plt.figure(figsize=(10,10))
for i in range(15):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(x_train_reshape[i], cmap='gray')
    plt.xlabel(col_names[y_train_fashion[i]])
plt.show()

In [ ]:
#Names of numbers in the dataset in order
col_names = ['Zero','One','Two','Three','Four','Five','Six','Seven','Eight','Nine']

#Visualizing the digits
plt.figure(figsize=(10,10))
for i in range(15):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(X_train_digit[i], cmap='gray')
    plt.xlabel(col_names[y_train_digit[i]])
plt.show()

In [ ]:
def visualize_input(img, ax):
    ax.imshow(img, cmap='summer')
    width, height = img.shape
    thresh = img.max()/2.5
    for x in range(width):
        for y in range(height):
            ax.annotate(str(round(img[x][y],2)), xy=(y,x),
                        horizontalalignment='center',
                        verticalalignment='center',
                        color='white' if img[x][y]<thresh else 'black')

In [ ]:
#Visualizing for digit MNIST
fig = plt.figure(figsize = (12,12)) 
ax = fig.add_subplot(111)
visualize_input(X_train_digit[1], ax)
plt.show()

In [ ]:
#Visualizing for Fashion MNIST
fig = plt.figure(figsize = (12,12)) 
ax = fig.add_subplot(111)
visualize_input(x_train_reshape[1], ax)
plt.show()

In [ ]:
X_train_digit = X_train_digit.reshape(60000, 784)
X_test_digit = X_test_digit.reshape(10000, 784)

In [ ]:
#Encoding Digit MNIST Labels
y_train_digit = to_categorical(y_train_digit, num_classes=10)

y_test_digit = to_categorical(y_test_digit, num_classes=10)


#Encoding Fashion MNIST Labels
y_train_fashion = to_categorical(y_train_fashion, num_classes=10)

y_test_fashion = to_categorical(y_test_fashion, num_classes=10)

# Fitting the Fashion Model

In [ ]:
image_shape = 784

In [ ]:
#Creating base neural network
model2 = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(image_shape,)),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(24, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(24, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(10,activation='softmax'),
])

#Compiling the model
model2.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics = ['accuracy'])

#Fitting the model
model2.fit(X_train_fashion, y_train_fashion, batch_size=100, epochs=1)

In [ ]:
model2.summary()

In [ ]:
model2.save(fr'./saved_models/ann.keras')

# Profiling the model

In [ ]:
%load_ext line_profiler

In [ ]:
def profile_model(model, x):
    x = model.layers[0](x) # Dense
    x = model.layers[1](x) # Dropout
    x = model.layers[2](x) # Batch Normalization
    x = model.layers[3](x) # Dense
    x = model.layers[4](x) # Dropout
    x = model.layers[5](x) # Batch Normalization
    x = model.layers[6](x) # Dense
    x = model.layers[7](x) # Dropout
    x = model.layers[8](x) # Batch Normalization
    x = model.layers[9](x) # Dense

In [ ]:
from profile_reader import read_lprun, read_mprun

In [ ]:
%lprun -T ./profiles/lp/ann_layer1.txt -f profile_model profile_model(model2, np.float_(X_test_fashion))

In [ ]:
read_lprun('ann')

In [ ]:
%%file mp_ann.py
from keras.models import Sequential
from keras.layers import Dense,Dropout,BatchNormalization
from memory_profiler import profile

@profile(precision=10)
def mp_ann_1(image_shape):
	model = Sequential()
	model.add(Dense(128, activation='relu', input_shape=(image_shape,)))
	model.add(Dropout(0.3))
	model.add(BatchNormalization())
	model.add(Dense(24, activation='relu'))
	model.add(Dropout(0.3))
	model.add(BatchNormalization())
	model.add(Dense(24, activation='relu'))
	model.add(Dropout(0.3))
	model.add(BatchNormalization())
	model.add(Dense(10,activation='softmax'))


In [ ]:
%load_ext memory_profiler

In [ ]:
from mp_ann import mp_ann_1
import sys

In [ ]:
stdout = sys.stdout
sys.stdout = open('./profiles/mp/ann_layer1.txt', 'w')
mp_ann_1(image_shape)
sys.stdout = stdout

In [ ]:
read_mprun('ann')